In [1]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [2]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [3]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [4]:
df=pd.read_sql('SELECT * FROM Purchasing.ShipMethod',sourceConn,index_col="ShipMethodID")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# df = pd.read_sql('SELECT * FROM Purchasing.[PurchaseOrderHeader]',sourceConn)
# df['ModifiedDate'] =pd.to_datetime(df['ModifiedDate']).dt.date
# date=df['ModifiedDate'].drop_duplicates().reset_index(drop=True)
# print(date)

# Extract data from source DB to pandas DataFrames

In [5]:
Tables= [('Purchasing.ShipMethod','ShipMethodID'),('Production.Product','ProductID'),('Purchasing.Vendor','BusinessEntityID'),('Purchasing.PurchaseOrderHeader','PurchaseOrderID'),
         ('Purchasing.PurchaseOrderDetail',['PurchaseOrderID','PurchaseOrderDetailID']), 
         ('Person.Person','BusinessEntityID'),('Sales.SalesTerritory','TerritoryID'),
         ('Person.CountryRegion','CountryRegionCode'),('Sales.SalesOrderHeader','SalesOrderID'),('Sales.SalesOrderDetail',['SalesOrderID','SalesOrderDetailID'])
        ]

In [ ]:
# df=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')
# df


In [6]:
dfs={}
for item in Tables:
    dfs[item[0]]=pd.read_sql(f'SELECT * FROM {item[0]}',sourceConn,index_col=item[1])
dfs['HumanResources.Employee']=pd.read_sql('SELECT BusinessEntityID FROM HumanResources.Employee',sourceConn,index_col='BusinessEntityID')


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas

## Transform staged data

In [10]:
#columns to extract in each table
Columns= [('Purchasing.ShipMethod','Name'),('Production.Product',['Name','ProductLine']),('Purchasing.Vendor',['Name','CreditRating']),
         ('Purchasing.PurchaseOrderHeader',['Status','EmployeeID','VendorID','ShipMethodID','OrderDate','ShipDate']),
         ('Purchasing.PurchaseOrderDetail',['DueDate','OrderQty','ProductID','LineTotal','ReceivedQty','RejectedQty','StockedQty']), 
         ('Person.Person',['FirstName','MiddleName','LastName','Suffix']),('Sales.SalesTerritory',['Name','CountryRegionCode','Group']),
         ('Person.CountryRegion','Name'),
         ('Sales.SalesOrderHeader',['OrderDate','DueDate','ShipDate','Status','OnlineOrderFlag','TerritoryID','ShipMethodID']),
         ('Sales.SalesOrderDetail',['OrderQty','ProductID','LineTotal'])
        ]

In [8]:
df=pd.DataFrame(dfs['Production.Product'][['Name','Color','Size','Class','Style']])
df

,Name,Color,Size,Class,Style
ProductID,,,,,
1,Adjustable Race,None,None,None,None
2,Bearing Ball,None,None,None,None
3,BB Ball Bearing,None,None,None,None
4,Headset Ball Bearings,None,None,None,None
316,Blade,None,None,None,None
...,...,...,...,...,...
995,ML Bottom Bracket,None,None,M,None
996,HL Bottom Bracket,None,None,H,None
997,"Road-750 Black, 44",Black,44,L,U


In [11]:
dfst={}
for item in Columns:
    dfst[item[0]]=dfs[item[0]][item[1]]



In [13]:
dfst['HumanResources.Employee']=dfs['HumanResources.Employee']

In [25]:
df=pd.merge(dfst['HumanResources.Employee'],dfst['Person.Person'],how='left',on='BusinessEntityID')
df['Name']= df['FirstName'].map(str)+' '+df['MiddleName'].map(str)+'. '+df['LastName'].map(str)
df.sort_index()

,FirstName,MiddleName,LastName,Suffix,Name
BusinessEntityID,,,,,
1,Ken,J,Sánchez,None,Ken J. Sánchez
2,Terri,Lee,Duffy,None,Terri Lee. Duffy
3,Roberto,None,Tamburello,None,Roberto None. Tamburello
4,Rob,None,Walters,None,Rob None. Walters
5,Gail,A,Erickson,None,Gail A. Erickson
...,...,...,...,...,...
286,Lynn,N,Tsoflias,None,Lynn N. Tsoflias
287,Amy,E,Alberts,None,Amy E. Alberts
288,Rachel,B,Valdez,None,Rachel B. Valdez


In [24]:
for index, row in df.iterrows():
      cursor.execute('''INSERT INTO dbo.DimEmployee (EmployeeID,Name) values(?,?)''',index, row.Name)
      
cursor.commit()
